In [ ]:
import pandas as pd
import numpy as np
from sklearn.metrics import accuracy_score
from sklearn.metrics import precision_score
from sklearn.metrics import recall_score
from sklearn.metrics import f1_score

In [ ]:
from google.colab import files
uploaded = files.upload()

Saving Labelled dataset (2).csv to Labelled dataset (2) (1).csv


In [ ]:
import io
df =  pd.read_csv(io.StringIO(uploaded['Labelled dataset (2).csv'].decode('utf-8')))
df.head()

,Decision,Sentiment,hotel,polarity,source,text,anger,fear,anticipation,trust,surprise,sadness,joy,disgust,positive,negative,n_count,wrd_count
0,truthful,1,conrad,positive,TripAdvisor,We stayed for a one night getaway with family ...,0.009524,0.009524,0.019048,0.019048,0.009524,0.019048,0.038095,0.000000,0.057143,0.028571,0.287037,2
1,truthful,1,hyatt,positive,TripAdvisor,Triple A rate with upgrade to view room was le...,0.000000,0.000000,0.000000,0.000000,0.000000,0.000000,0.000000,0.000000,0.066667,0.000000,0.400000,2
2,truthful,1,hyatt,positive,TripAdvisor,This comes a little late as I'm finally catchi...,0.004831,0.004831,0.024155,0.043478,0.004831,0.014493,0.028986,0.004831,0.057971,0.019324,0.236967,8
3,truthful,1,omni,positive,TripAdvisor,The Omni Chicago really delivers on all fronts...,0.000000,0.007874,0.007874,0.031496,0.007874,0.007874,0.015748,0.007874,0.039370,0.015748,0.356589,3
4,truthful,1,hyatt,positive,TripAdvisor,I asked for a high floor away from the elevato...,0.000000,0.000000,0.027778,0.055556,0.027778,0.000000,0.041667,0.000000,0.069444,0.000000,0.205479,4


In [ ]:
df2 = df.reindex(np.random.permutation(df.index))
print(df2)


       Decision  Sentiment             hotel  ...  negative   n_count wrd_count
513   deceptive          0     knickerbocker  ...  0.000000  0.223684         4
140    truthful          1          sheraton  ...  0.000000  0.259494         6
1280  deceptive          0     knickerbocker  ...  0.022901  0.213740         5
796   deceptive          0            amalfi  ...  0.000000  0.353535         4
606   deceptive          0           talbott  ...  0.000000  0.287356         2
...         ...        ...               ...  ...       ...       ...       ...
919    truthful          1     knickerbocker  ...  0.000000  0.206612         3
884    truthful          1          homewood  ...  0.020979  0.212329         3
726   deceptive          0  intercontinental  ...  0.009804  0.254902         4
913    truthful          1          sheraton  ...  0.009615  0.274882         4
929    truthful          1          sheraton  ...  0.009524  0.212963         4

[1600 rows x 18 columns]


In [ ]:


#col_names1 = ['Decision','Sentiment','anger','fear','anticipation','trust','surprise','sadness','joy','disgust','positive','negative','n count','wrd count']

#df3 = pd.DataFrame({'Decision':df2['Decision'],'Sentiment':df2['Sentiment'],'anger':df2['anger'],'fear':df2['fear'],'anticipation':df2['anticipation'],'trust':df2['trust'],'surprise':df2['surprise'],'sadness':df2['sadness'],'joy':df2['joy'],'disgust':df2['disgust'],'positive':df2['positive'],'negative':df2['negative'],'n_count':df2['n_count'],'wrd_count':df2['wrd_count']},columns = col_names1)

df3 = df2.copy()
# df3.drop(["hotel","polarity","source","text"],axis=1,inplace=True)
# print(df3)
# df3.to_csv('dataset.csv')

col_names = ['text','labels']
df2 = pd.DataFrame({'text':df2['text'], 'labels':df2['Sentiment']}, columns=col_names)





train_x = df2['text'].to_numpy()
labels = df2['labels'].to_numpy()



split_frac1 = 0.8

idx1 = int(len(train_x) * split_frac1)
texts_train, texts_test = train_x[:idx1], train_x[idx1:]
y_train, y_test = labels[:idx1], labels[idx1:]


print(len(texts_train))
print(len(y_train))
print(len(texts_test))
print(len(y_test))


1280
1280
320
320


In [ ]:
col_names = ['labels','text']
df_trn = pd.DataFrame({'text':texts_train, 'labels':y_train}, columns=col_names)
df_val = pd.DataFrame({'text':texts_test, 'labels':y_test}, columns=col_names)

In [ ]:
print(df_trn)

      labels                                               text
0          1  I stayed in this hotel during a business trip,...
1          0  We booked this trip several months in advance ...
2          1  We live down the block from the hotel. We arra...
3          0  My stay at the Hyatt Regency was an experience...
4          1  This is a great hotel, in a great location, in...
...      ...                                                ...
1275       0  I was very disappointed with this hotel. The f...
1276       0  I stayed at the Affnia hotel in Chicago and wa...
1277       1  Other reviews complain about the small rooms. ...
1278       0  From the moment I walked in I was impressed. T...
1279       1  Just returned from a week in Chicago with the ...

[1280 rows x 2 columns]


In [ ]:
vocab_size = 20000  # Max number of different word, i.e. model input dimension
maxlen = 200  # Max number of words kept at the end of each text

In [ ]:
from keras.preprocessing.text import Tokenizer
from keras.preprocessing.sequence import pad_sequences
from sklearn.pipeline import TransformerMixin
from sklearn.base import BaseEstimator

class TextsToSequences(Tokenizer, BaseEstimator, TransformerMixin):
    """ Sklearn transformer to convert texts to indices list """
    def __init__(self,  **kwargs):
        super().__init__(**kwargs)
        
    def fit(self, texts, y=None):
        self.fit_on_texts(texts)
        return self
    
    def transform(self, texts, y=None):
        return np.array(self.texts_to_sequences(texts))
        
sequencer = TextsToSequences(num_words=vocab_size)

In [ ]:
# object = TextsToSequences()
# object.fit("The cute cat")
# a = object.transform("The cute cat")
# print(a)



[list([1]) list([4]) list([2]) list([]) list([3]) list([5]) list([1])
 list([2]) list([]) list([3]) list([6]) list([1])]


In [ ]:
class Padder(BaseEstimator, TransformerMixin):
    """ Pad and crop uneven lists to the same length. 
    Only the end of lists longern than the maxlen attribute are
    kept, and lists shorter than maxlen are left-padded with zeros
    
    Attributes
    ----------
    maxlen: int
        sizes of sequences after padding
    max_index: int
        maximum index known by the Padder, if a higher index is met during 
        transform it is transformed to a 0
    """
    def __init__(self, maxlen=500):
        self.maxlen = maxlen
        self.max_index = None
        
    def fit(self, X, y=None):
        self.max_index = pad_sequences(X, maxlen=self.maxlen).max()
        return self
    
    def transform(self, X, y=None):
        X = pad_sequences(X, maxlen=self.maxlen)
        X[X > self.max_index] = 0
        return X

padder = Padder(maxlen)

In [ ]:
import numpy as np
from keras.models import Sequential
from tensorflow.keras.layers import Dense, Embedding, Bidirectional, LSTM, Dropout
from keras.wrappers.scikit_learn import KerasClassifier
from sklearn.pipeline import make_pipeline

batch_size = 64
max_features = vocab_size + 1

def create_model(max_features):
    """ Model creation function: returns a compiled Bidirectional LSTM"""
    model = Sequential()
    model.add(Embedding(max_features, 128))
    model.add(Dropout(rate=0.25))
    model.add(LSTM(128,return_sequences=True))
    model.add(Dropout(rate=0.25))
    
    model.add(LSTM(128,return_sequences=True))
    model.add(Dropout(rate=0.25))
    
    model.add(LSTM(128))
    model.add(Dropout(rate=0.25))
    model.add(Dense(1,activation='sigmoid'))
    #model.add(Dropout(rate=0.25))
    model.compile('adam', 'binary_crossentropy', metrics=['accuracy'])
    return model

# Use Keras Scikit-learn wrapper to instantiate a LSTM with all methods
# required by Scikit-learn for the last step of a Pipeline
sklearn_lstm = KerasClassifier(build_fn=create_model, epochs=10, batch_size=batch_size, 
                               max_features=max_features, verbose=True)



# 

# Build the Scikit-learn pipeline
pipeline = make_pipeline(sequencer, padder, sklearn_lstm)

pipeline.fit(texts_train, y_train)



Epoch 1/10
20/20 [==============================] - 1s 61ms/step - loss: 0.6835 - accuracy: 0.5352
Epoch 2/10
20/20 [==============================] - 1s 52ms/step - loss: 0.4687 - accuracy: 0.7898
Epoch 3/10
20/20 [==============================] - 1s 52ms/step - loss: 0.2066 - accuracy: 0.9375
Epoch 4/10
20/20 [==============================] - 1s 52ms/step - loss: 0.1772 - accuracy: 0.9422
Epoch 5/10
20/20 [==============================] - 1s 52ms/step - loss: 0.1117 - accuracy: 0.9656
Epoch 6/10
20/20 [==============================] - 1s 53ms/step - loss: 0.0459 - accuracy: 0.9891
Epoch 7/10
20/20 [==============================] - 1s 50ms/step - loss: 0.0206 - accuracy: 0.9930
Epoch 8/10
20/20 [==============================] - 1s 54ms/step - loss: 0.0090 - accuracy: 0.9984
Epoch 9/10
20/20 [==============================] - 1s 54ms/step - loss: 0.0038 - accuracy: 0.9992
Epoch 10/10
20/20 [==============================] - 1s 52ms/step - loss: 0.0081 - accuracy: 0.9977


Pipeline(memory=None,
         steps=[('textstosequences', TextsToSequences()),
                ('padder', Padder(maxlen=200)),
                ('kerasclassifier',
                 <tensorflow.python.keras.wrappers.scikit_learn.KerasClassifier object at 0x7f1a92ac45c0>)],
         verbose=False)

In [ ]:
# !pip show tensorflow


Name: tensorflow
Version: 2.3.0
Summary: TensorFlow is an open source machine learning framework for everyone.
Home-page: https://www.tensorflow.org/
Author: Google Inc.
Author-email: packages@tensorflow.org
License: Apache 2.0
Location: /usr/local/lib/python3.6/dist-packages
Requires: termcolor, opt-einsum, grpcio, gast, keras-preprocessing, h5py, wheel, google-pasta, tensorflow-estimator, wrapt, absl-py, astunparse, tensorboard, protobuf, six, scipy, numpy
Required-by: fancyimpute


In [ ]:
from sklearn import metrics
print('Computing predictions on test set...')
y_preds = pipeline.predict(texts_test)

print(y_preds)



#np.savetxt('lstm.csv',df3, fmt= '%d')

print('Test accuracy: {:.2f} %'.format(100*metrics.accuracy_score(y_preds, y_test)))
# # accuracy: (tp + tn) / (p + n)
# accuracy = accuracy_score(y_test, y_preds)
# print('Accuracy: %f' % accuracy)
# # precision tp / (tp + fp)
# precision = precision_score(y_test, y_preds)
# print('Precision: %f' % precision)
# # recall: tp / (tp + fn)
# recall = recall_score(y_test, y_preds)
# print('Recall: %f' % recall)
# # f1: 2 tp / (2 tp + fp + fn)
# f1 = f1_score(y_test, y_preds)
# print('F1 score: %f' % f1)

In [ ]:
from sklearn import metrics

y_preds = pipeline.predict(train_x)


# c=0
# d=0
# for i in range(320):
#   if(y_preds[i]==0):
#     c = c+1
#   else:
#     d = d+1 

df4 = y_preds

np.savetxt('lstm.csv',df4, fmt = '%d')

#print('Test accuracy: {:.2f} %'.format(100*metrics.accuracy_score(y_preds, labels)))

25/25 [==============================] - 0s 14ms/step


In [ ]:

df5 = pd.DataFrame(data=df4,columns=["lstm"])


In [ ]:
df6 = df3.copy()
df3.columns

Index(['Decision', 'Sentiment', 'hotel', 'polarity', 'source', 'text', 'anger',
       'fear', 'anticipation', 'trust', 'surprise', 'sadness', 'joy',
       'disgust', 'positive', 'negative', 'n_count', 'wrd_count'],
      dtype='object')

In [ ]:
df3 = pd.concat([df3, df5], axis=1)

In [ ]:
df3.columns

Index(['Decision', 'Sentiment', 'hotel', 'polarity', 'source', 'text', 'anger',
       'fear', 'anticipation', 'trust', 'surprise', 'sadness', 'joy',
       'disgust', 'positive', 'negative', 'n_count', 'wrd_count', 'lstm'],
      dtype='object')

In [ ]:
df3.to_csv('final_dataset.csv')